# A3. Loading AP Sensing files
This example loads AP sensing files. Only single-ended files are currently supported. Just like with Silixa's devices, the AP Sensing data is in .xml files

In [ ]:
import os
import glob

from dtscalibration import read_apsensing_files

The example data files are located in `./python-dts-calibration/tests/data`.

In [ ]:
filepath = os.path.join('..', '..', 'tests', 'data', 'ap_sensing')
print(filepath)

In [ ]:
filepathlist = sorted(glob.glob(os.path.join(filepath, '*.xml')))
filenamelist = [os.path.basename(path) for path in filepathlist]

for fn in filenamelist:
    print(fn)

We will simply load in the .xml files

In [ ]:
ds = read_apsensing_files(directory=filepath)

The object tries to gather as much metadata from the measurement files as possible (temporal and spatial coordinates, filenames, temperature probes measurements). All other configuration settings are loaded from the first files and stored as attributes of the `DataStore`.

Calibration follows as usual (see the other notebooks).

In [ ]:
print(ds)